# Import Lib

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import (
    Dense, Dropout, LayerNormalization, 
    MultiHeadAttention, Conv2D, Rescaling
)
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from PIL import ImageFile
import pandas as pd
from tqdm import tqdm
import numpy as np
import os

# Submission of models to meet the imagenet supported standards

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_LAYERS = 8
EMBED_DIM = 384
MLP_DIM = 4 * EMBED_DIM
NUM_HEADS = 8
DROPOUT_RATE = 0.1
NUM_CLASSES = 1000

IMG_HEIGHT, IMG_WIDTH = (IMAGE_SIZE, IMAGE_SIZE)
IMG_CHANNELS = 3
BATCH_SIZE = 32

In [ ]:
class EncoderBlock(Model):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout_rate=0.1, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.embed_dim, self.num_heads, self.mlp_dim, self.dropout_rate = embed_dim, num_heads, mlp_dim, dropout_rate
        self.norm1 = LayerNormalization(epsilon=1e-6, name="norm1")
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout_rate, name="multi_head_attention")
        self.dropout_mha_output = Dropout(dropout_rate)
        self.norm2 = LayerNormalization(epsilon=1e-6, name="norm2")
        self.mlp = Sequential([
            Dense(mlp_dim, activation='gelu', name="mlp_dense_1"),
            Dropout(dropout_rate),
            Dense(embed_dim, name="mlp_dense_2"),
            Dropout(dropout_rate)
        ], name="mlp_block")

    def call(self, inputs, training=False):
        x_norm1 = self.norm1(inputs)
        attn_output = self.mha(query=x_norm1, value=x_norm1, key=x_norm1, training=training)
        attn_output_dropped = self.dropout_mha_output(attn_output, training=training)
        x_res1 = inputs + attn_output_dropped
        x_norm2 = self.norm2(x_res1)
        mlp_output = self.mlp(x_norm2, training=training)
        x_res2 = x_res1 + mlp_output
        return x_res2

    def get_config(self):
        config = super(EncoderBlock, self).get_config()
        config.update({'embed_dim': self.embed_dim, 'num_heads': self.num_heads, 'mlp_dim': self.mlp_dim, 'dropout_rate': self.dropout_rate})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

class VisionTransformer(Model):
    def __init__(self, image_size=IMAGE_SIZE, num_classes=NUM_CLASSES, patch_size=PATCH_SIZE, embed_dim=EMBED_DIM,
                 num_heads=NUM_HEADS, num_layers=NUM_LAYERS, mlp_dim=MLP_DIM, dropout_rate=DROPOUT_RATE, **kwargs):
        super(VisionTransformer, self).__init__(**kwargs)
        if isinstance(image_size, int): image_size = (image_size, image_size)
        if isinstance(patch_size, int): patch_size = (patch_size, patch_size)
        self.image_size, self.num_classes, self.patch_size = image_size, num_classes, patch_size
        self.embed_dim, self.num_heads, self.num_layers = embed_dim, num_heads, num_layers
        self.mlp_dim, self.dropout_rate = mlp_dim, dropout_rate
        self.num_patches = (image_size[0] // patch_size[0]) * (image_size[1] // patch_size[1])
        self.cls_token = self.add_weight(name="cls_token", shape=[1, 1, embed_dim], initializer=tf.keras.initializers.RandomNormal(stddev=0.02), trainable=True)
        self.pos_embed = self.add_weight(name="position_embedding", shape=[1, self.num_patches + 1, embed_dim], initializer=tf.keras.initializers.RandomNormal(stddev=0.02), trainable=True)
        self.pos_dropout = Dropout(dropout_rate)
        self.patch_embed = Conv2D(filters=embed_dim, kernel_size=patch_size, strides=patch_size, padding='valid', name="patch_embed")
        self.encoder_layers = [EncoderBlock(embed_dim, num_heads, mlp_dim, dropout_rate) for _ in range(num_layers)]
        self.norm_head = LayerNormalization(epsilon=1e-6, name="head_norm")
        self.head = Dense(num_classes, activation='softmax', name="classification_head")

    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        x = self.patch_embed(inputs)
        x = tf.reshape(x, (batch_size, -1, self.embed_dim))
        cls_tokens = tf.tile(self.cls_token, [batch_size, 1, 1])
        x = tf.concat([cls_tokens, x], axis=1)
        x = x + self.pos_embed
        x = self.pos_dropout(x, training=training)
        for encoder in self.encoder_layers:
            x = encoder(x, training=training)
        cls_token_output = self.norm_head(x[:, 0], training=training)
        logits = self.head(cls_token_output, training=training)
        return logits

    def get_config(self):
        config = super(VisionTransformer, self).get_config()
        config.update({'image_size': self.image_size, 'num_classes': self.num_classes, 'patch_size': self.patch_size, 'embed_dim': self.embed_dim, 'num_heads': self.num_heads, 'num_layers': self.num_layers, 'mlp_dim': self.mlp_dim, 'dropout_rate': self.dropout_rate})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, warmup_steps, alpha=0.0, name=None):
        super().__init__()
        self.name, self.initial_learning_rate, self.decay_steps, self.warmup_steps, self.alpha = name, initial_learning_rate, decay_steps, warmup_steps, alpha
        self.cosine_decay_schedule = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=self.initial_learning_rate, decay_steps=self.decay_steps - self.warmup_steps, alpha=self.alpha)

    def __call__(self, step):
        step_float = tf.cast(step, tf.float32)
        warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)
        def warmup_fn(): return (self.initial_learning_rate / warmup_steps_float) * step_float
        def cosine_decay_fn(): return self.cosine_decay_schedule(step_float - warmup_steps_float)
        return tf.cond(step_float < warmup_steps_float, warmup_fn, cosine_decay_fn)

    def get_config(self):
        return {"initial_learning_rate": self.initial_learning_rate, "decay_steps": self.decay_steps, "warmup_steps": self.warmup_steps, "alpha": self.alpha, "name": self.name}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


try:
    idx_to_ilsvrc_id_df = pd.read_csv("https://github.com/TheoBourdais/ImageNetSubmission/raw/main/src/idx_to_ILSVRC_ID.csv")
    idx_to_ilsvrc_id = {idx: ilsvrc_id for idx, ilsvrc_id in zip(idx_to_ilsvrc_id_df["idx"], idx_to_ilsvrc_id_df["ILSVRC_ID"])}
except Exception as e:
    print(f"ไม่สามารถดาวน์โหลดไฟล์ Class Mappings ได้: {e}"); exit()

rescale_layer_only = Rescaling(1./255)

def preprocess_image_tf(image_path, target_height=IMG_HEIGHT, target_width=IMG_WIDTH):
    img_bytes = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.resize(img, [target_height, target_width])
    return image_path, rescale_layer_only(img)

def create_tf_dataset(image_folder_path, batch_size):
    image_paths = sorted(tf.io.gfile.glob(os.path.join(image_folder_path, "**", "*.JPEG")))
    if not image_paths:
        for ext in ["*.jpeg", "*.jpg"]:
            image_paths = sorted(tf.io.gfile.glob(os.path.join(image_folder_path, "**", ext)))
            if image_paths: break
    if not image_paths: raise ValueError(f"ไม่พบไฟล์รูปภาพใน: {image_folder_path}")
    print(f"พบรูปภาพทั้งหมด {len(image_paths)} รูป")
    filename_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    image_dataset = filename_dataset.map(lambda x: preprocess_image_tf(x), num_parallel_calls=tf.data.AUTOTUNE)
    return image_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

def get_test_submission_tf(model, tf_dataloader):
    all_filenames, all_predictions_np = [], []
    print("กำลังประมวลผลรูปภาพ...")
    for batch_paths, batch_images in tqdm(tf_dataloader):
        outputs = model(batch_images, training=False)
        _, predicted_indices = tf.math.top_k(outputs, k=5)
        all_filenames.extend([os.path.basename(p.numpy().decode('utf-8')) for p in batch_paths])
        all_predictions_np.extend(predicted_indices.numpy())
    mapped_predictions = [[idx_to_ilsvrc_id.get(i, "n00000000") for i in row] for row in all_predictions_np]
    submission_df = pd.DataFrame(mapped_predictions, columns=[f"choice {i}" for i in range(1, 6)], index=all_filenames)
    return submission_df.sort_index()

if __name__ == '__main__':
    print(f"TensorFlow Version: {tf.__version__}")
    
    keras_model_path = "/media/capybara/Data/dataset_vit/codeVIT/best_vit_model_1000_classes.keras"
    print(f"กำลังโหลดโมเดลจาก: {keras_model_path}")

    custom_objects = {
        "EncoderBlock": EncoderBlock,
        "VisionTransformer": VisionTransformer,
        "WarmupCosineDecay": WarmupCosineDecay
    }
    
    try:
        model = load_model(keras_model_path, custom_objects=custom_objects)
        print("โหลดโมเดลสำเร็จ")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการโหลดโมเดล: {e}"); exit()

    model.summary()

    validation_set_path = "/media/capybara/Data/dataset_vit/testdata"
    
    print(f"กำลังเตรียม TensorFlow dataset จาก: {validation_set_path}")
    test_tf_dataset = create_tf_dataset(validation_set_path, batch_size=BATCH_SIZE)

    print("กำลังสร้างไฟล์ submission...")
    submission_df = get_test_submission_tf(model, test_tf_dataset)

    submission_file_path = "submission_custom_model.txt"
    print(f"กำลังบันทึกไฟล์ submission ไปที่: {submission_file_path}")
    submission_df.to_csv(submission_file_path, index=False, header=False, sep=" ")
    print(f"สร้างไฟล์ '{submission_file_path}' สำเร็จ")
    print("เสร็จสิ้น")

TensorFlow Version: 2.19.0
กำลังโหลดโมเดลจาก: /media/capybara/Data/dataset_vit/codeVIT/best_vit_model_1000_classes.keras


I0000 00:00:1749187385.399184   25183 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7209 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


โหลดโมเดลสำเร็จ


Model: "vision_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_embed (Conv2D)            │ (1, 14, 14, 384)       │       295,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block (EncoderBlock)    │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_1 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_2 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_3 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_4 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_5 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_6 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_7 (EncoderBlock)  │ ?                      │     1,774,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head_norm (LayerNormalization)  │ (1, 384)               │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_head (Dense)     │ (1, 1000)              │       385,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Optimizer params: 29,905,617 (114.08 MB)

กำลังเตรียม TensorFlow dataset จาก: /media/capybara/Data/dataset_vit/testdata
พบรูปภาพทั้งหมด 100000 รูป
กำลังสร้างไฟล์ submission...
กำลังประมวลผลรูปภาพ...


100%|██████████| 3125/3125 [14:18<00:00,  3.64it/s]


กำลังบันทึกไฟล์ submission ไปที่: submission_custom_model.txt
สร้างไฟล์ 'submission_custom_model.txt' สำเร็จ
เสร็จสิ้น


: 